# Building an NLP Pipeline Class

For the pair problem today, you're going to work on building a class that manages your NLP needs. The goal is to build something that takes in a bunch of classes, and can spit out the cleaned text as a vector. I'll get you started with a template.

In [9]:
class nlp_pipe:
    
    def __init__(self, vectorizer, cleaning_function, tokenizer, stemmer):
        self.vectorizer = vectorizer
        self.cleaning_function = cleaning_function
        self.tokenizer = tokenizer
        self.stemmer = stemmer
    
    def fit(self, text_to_fit_on):
        pass
    
    def transfrom(self, text_to_clean):
        pass

As a quick note, if you want to pass a function into a class you can do so like this:

In [10]:
def print_the_word_bob_three_times():
    for i in range(3):
        print('bob')
        
        
class this_is_an_example:
    
    def __init__(self, function_input):
        self.function_to_run = function_input
        
    def do_the_thing(self):
        self.function_to_run()

In [11]:
example = this_is_an_example(print_the_word_bob_three_times)
# Note that when I put the function in, I didn't invoke it with the parens!

In [12]:
example.do_the_thing()

bob
bob
bob


So what I want is the ability to do something like:

```python
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer

nlp = nlp_pipe(CountVectorizer(), simple_cleaning_function_i_made, TreebankWordTokenizer(), PorterStemmer())
nlp.fit(train_corpus)
nlp.transform(test_corpus)
```
Which should return the test corpus in its vectorizer format.

# Solution!

In [70]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle


class nlp_preprocessor:
   
    def __init__(self, vectorizer=CountVectorizer(), tokenizer=None, 
                 cleaning_function=None, stemmer=None):
        """
        A class for pipelining our data in NLP problems. The user provides a series of 
        tools, and this class manages all of the training, transforming, and modification
        of the text data.
        ---
        Inputs:
        vectorizer: the model to use for vectorization of text data
        tokenizer: The tokenizer to use, if none defaults to split on spaces
        cleaning_function: how to clean the data, if None, defaults to the in built class
        """
        if not tokenizer:
            tokenizer = self.splitter
        if not cleaning_function:
            cleaning_function = self.clean_text
        self.stemmer = stemmer
        self.tokenizer = tokenizer
     #   self.model = model
        self.cleaning_function = cleaning_function
        self.vectorizer = vectorizer
        self._is_fit = False
        
    def splitter(self, text):
        """
        Default tokenizer that splits on spaces naively
        """
        return text.split(' ')
        
    def clean_text(self, text, tokenizer, stemmer):
        """
        A naive function to lowercase all works can clean them quickly.
        This is the default behavior if no other cleaning function is specified
        """
        cleaned_text = []
        for post in text:
            cleaned_words = []
            for word in tokenizer(post):
                low_word = word.lower()
                if stemmer:
                    low_word = stemmer.stem(low_word)
                cleaned_words.append(low_word)
            cleaned_text.append(' '.join(cleaned_words))
        return cleaned_text
    
    def fit1(self, text):
        """
        Cleans the data and then fits the vectorizer with
        the user provided text
        """
        clean_text = self.cleaning_function(text, self.tokenizer, self.stemmer)
        print(clean_text)
        self.vectorizer.fit(clean_text)
        self._is_fit = True
        return self.vectorizer.fit(clean_text)

    
    def fit(self, text):
        """
        Cleans the data and then fits the vectorizer with
        the user provided text
        """
        clean_text = self.cleaning_function(text, self.tokenizer, self.stemmer)
        self.vectorizer.fit(clean_text)
        self._is_fit = True
        return self.vectorizer.fit(clean_text)
       
    def transform(self, text):
        """
        Cleans any provided data and then transforms the data into
        a vectorized format based on the fit function. Returns the
        vectorized form of the data.
        """
        if not self._is_fit:
            raise ValueError("Must fit the models before transforming!")
        clean_text = self.cleaning_function(text, self.tokenizer, self.stemmer)
        return self.vectorizer.transform(clean_text)
    
    def save_pipe(self, filename):
        """
        Writes the attributes of the pipeline to a file
        allowing a pipeline to be loaded later with the
        pre-trained pieces in place.
        """
        if type(filename) != str:
            raise TypeError("filename must be a string")
        pickle.dump(self.__dict__, open(filename+".mdl",'wb'))
        
    def load_pipe(self, filename):
        """
        Writes the attributes of the pipeline to a file
        allowing a pipeline to be loaded later with the
        pre-trained pieces in place.
        """
        if type(filename) != str:
            raise TypeError("filename must be a string")
        if filename[-4:] != '.mdl':
            filename += '.mdl'
        self.__dict__ = pickle.load(open(filename,'rb'))

In [71]:
corpus = ['BOB the builder', 'is a strange', 'caRtoon type thing']


In [72]:
def simple_cleaning_function_i_made(text, tokenizer, stemmer):
    cleaned_text = []
    for post in text:
        cleaned_words = []
        for word in tokenizer(post):
            low_word = word.lower()
            if stemmer:
                low_word = stemmer.stem(low_word)
            cleaned_words.append(low_word)
        cleaned_text.append(' '.join(cleaned_words))
    return cleaned_text

In [73]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import TreebankWordTokenizer
from nltk.stem import PorterStemmer

nlp = nlp_preprocessor(vectorizer=TfidfVectorizer(min_df=0.3, max_df=0.8), 
                       cleaning_function=simple_cleaning_function_i_made, 
                       tokenizer=TreebankWordTokenizer().tokenize, stemmer=PorterStemmer())
nlp.fit(corpus)
vectorized_docs = nlp.transform(corpus).toarray()

In [75]:
nlp.fit1(corpus)

['bob the builder', 'is a strang', 'cartoon type thing']


TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=0.8, max_features=None, min_df=0.3,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words=None, strip_accents=None, sublinear_tf=False,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [ ]:
    def fit(self, text):
        """
        Cleans the data and then fits the vectorizer with
        the user provided text
        """
        clean_text = self.cleaning_function(text, self.tokenizer, self.stemmer)
        self.vectorizer.fit(clean_text)
        self._is_fit = True